In [82]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

pd.set_option('display.max_columns',None)
df = pd.read_csv(r"PLACEHOLDER.csv")

#add simplified EV Type column

for i in range(len(df)):
    if 'PHEV' in  df.loc[i,'Electric Vehicle Type']:
        df.loc[i, 'EV Type (Shortened)'] = 'PHEV'
    else:
        df.loc[i, 'EV Type (Shortened)'] = 'BEV'

#trim data down to rows that have mile range
        
df = df[df['Electric Range'] > 0.0].reset_index(drop=True)

df.drop(columns = ['VIN (1-10)','Base MSRP','DOL Vehicle ID','Vehicle Location','Electric Utility','2020 Census Tract'], inplace = True)

display(df.head(3))


,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Legislative District,EV Type (Shortened)
0,Kitsap,Seabeck,WA,98380.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,42.0,35.0,PHEV
1,Kitsap,Bremerton,WA,98312.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,151.0,35.0,BEV
2,King,Seattle,WA,98101.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266.0,43.0,BEV


In [120]:

#----------------------------------------------- create crosstabs -------------------------------------------------------------

pd.set_option('display.max_columns',None)

#crosstab City and Make
city = pd.crosstab(df.City,df.Make)

#get the total makes for each city and turn to dataframe
city_sum = city.sum(axis=1)
city_df = city_sum.reset_index()
city_df.columns = ['City', 'Sum of Makes']

#crosstab County and Make
county = pd.crosstab(df.County, df.Make)

#get the total makes for each city and turn to dataframe
county_sum = county.sum(axis=1)
county_df = county_sum.reset_index()
county_df.columns = ['County', 'Sum of Makes']

#crosstab Make and Model
make = pd.crosstab(df.Make,df.Model)

#get the total makes (sum and unique) for each city and turn to dataframe
model_sum = make.sum(axis=1)
model_sum_df = model_sum.reset_index()
model_sum_df.columns = ['Make', 'Sum of Models']
model_count = make.nunique(axis=1)
model_count_df = model_count.reset_index()
model_count_df.columns = ['Make', 'Count of Models']

make_type = pd.crosstab(df.Make,df['EV Type (Shortened)'])
mt_prop = make_type/len(df)
# display(mt_prop)

# display(city.head(),county.head(),make.head())
# display(city_df.head(),county_df.head(),model_sum_df.head(),model_count_df.head())

In [118]:
#-----------------------------------------One- Hot Encode Eligibility and EV Type columns --------------------------------------


df1 = pd.get_dummies(data = df,columns = ['Clean Alternative Fuel Vehicle (CAFV) Eligibility','EV Type (Shortened)'])

df1['Clean Alternative Fuel Vehicle (CAFV) Eligibility_Clean Alternative Fuel Vehicle Eligible'] = df1['Clean Alternative Fuel Vehicle (CAFV) Eligibility_Clean Alternative Fuel Vehicle Eligible'].astype('int')
df1['Clean Alternative Fuel Vehicle (CAFV) Eligibility_Not eligible due to low battery range'] = df1['Clean Alternative Fuel Vehicle (CAFV) Eligibility_Not eligible due to low battery range'].astype('int')
df1['EV Type (Shortened)_BEV'] = df1['EV Type (Shortened)_BEV'].astype('int')
df1['EV Type (Shortened)_PHEV'] = df1['EV Type (Shortened)_PHEV'].astype('int')

df1.rename(columns = {'Clean Alternative Fuel Vehicle (CAFV) Eligibility_Clean Alternative Fuel Vehicle Eligible':'CAFV Eligible',
                     'Clean Alternative Fuel Vehicle (CAFV) Eligibility_Not eligible due to low battery range':'CAFV Ineligible',
                     'EV Type (Shortened)_BEV':'BEV','EV Type (Shortened)_PHEV':'PHEV'},inplace=True)


display(df1.head())

,County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Electric Range,Legislative District,CAFV Eligible,CAFV Ineligible,BEV,PHEV
0,Kitsap,Seabeck,WA,98380.0,2023,TOYOTA,RAV4 PRIME,Plug-in Hybrid Electric Vehicle (PHEV),42.0,35.0,1,0,0,1
1,Kitsap,Bremerton,WA,98312.0,2018,NISSAN,LEAF,Battery Electric Vehicle (BEV),151.0,35.0,1,0,1,0
2,King,Seattle,WA,98101.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),266.0,43.0,1,0,1,0
3,King,Seattle,WA,98125.0,2014,NISSAN,LEAF,Battery Electric Vehicle (BEV),84.0,46.0,1,0,1,0
4,Thurston,Yelm,WA,98597.0,2017,CHEVROLET,BOLT EV,Battery Electric Vehicle (BEV),238.0,20.0,1,0,1,0


In [119]:
# ---------------------------------------- create tables based on encoded columns----------------------------------------------


make_cafv = df1.groupby('Make').agg({'CAFV Eligible':'sum','CAFV Ineligible':'sum'}).reset_index()
make_bev = df1.groupby('Make').agg({'BEV':'sum','PHEV':'sum'}).reset_index()

display(make_cafv.head())
display(make_bev.head())

,Make,CAFV Eligible,CAFV Ineligible
0,ALFA ROMEO,77,0
1,AUDI,733,1614
2,AZURE DYNAMICS,4,0
3,BENTLEY,0,5
4,BMW,3996,1892


,Make,BEV,PHEV
0,ALFA ROMEO,0,77
1,AUDI,733,1614
2,AZURE DYNAMICS,4,0
3,BENTLEY,0,5
4,BMW,567,5321


In [106]:
# ----------------------------------------- proportion of EV Makes in Washington ----------------------------------------------


df.Make.value_counts(normalize=True)

Make
TESLA                   0.280093
NISSAN                  0.115353
CHEVROLET               0.106455
TOYOTA                  0.077658
BMW                     0.064520
JEEP                    0.059315
KIA                     0.046176
FORD                    0.043382
CHRYSLER                0.040467
VOLVO                   0.036161
AUDI                    0.025718
HYUNDAI                 0.013347
VOLKSWAGEN              0.011769
MITSUBISHI              0.011155
HONDA                   0.009402
FIAT                    0.008547
PORSCHE                 0.008262
MAZDA                   0.007681
DODGE                   0.007287
MERCEDES-BENZ           0.005545
LEXUS                   0.004482
MINI                    0.003682
LINCOLN                 0.003276
SMART                   0.002707
JAGUAR                  0.002312
POLESTAR                0.001655
CADILLAC                0.000964
ALFA ROMEO              0.000844
LAND ROVER              0.000756
SUBARU                  0.000701
FISKE